## Stock share portfolio analyser 1.0.1

This script helps me to analyse my stock share portfolio. I always needed to automate this process. Before I used to download all data from the stock markets website. And build the calculator every time when I update my portfolio (buy or sell any stock). But today I don't need the whole process anymore. just download the data and upload it here. Script automaticly does all the calculation, loads actual prices for all shares and returns a result.

One more thing I wanted to do is to build program that will not jost analyse my portfolio, but itself get all the data of stock prices, analyse and give a prediction of future prices. Yes it will require a lot more time, knowledge, experience and strategies (machine learning, deep learning and other technologies). But I think this will be the first version of that programm.

#### import necessary python modules

In [5]:
import collections
import requests
from bs4 import BeautifulSoup as BS
import re

# load the portfolio
data = open('portfolio.20-09-20.csv')

def preprocess_data(data):
    floatptn = re.compile('\d*,?\d+\.\d+') # pattern to match float numbers like '23,1123.00'
    intptn = re.compile('^\d+,?\d+$') # pattern to match integers like '132,123,000'
    keys = []
    i = 0
    lines = []
    for line in data: # loop through the lines of data. one line contains data of one trade
        line = line.split(';')  # split columns
        if i > 0: # if line does not contain column names
            el = {}
            for c in range(len(line)): # loop through columns and match integer and float patterns
                if intptn.search(line[c]) != None:
                    val = int(line[c].replace(',', ''))
                elif floatptn.search(line[c]) != None:
                    val = float(line[c].replace(',', ''))
                else:
                    val = line[c]
                el[keys[c]] = val
            lines.append(el)
        else:
            keys = line # include column names here
            i+=1
    return lines # return preprocessed list of trades

# get acutal prices for single share
def get_prices(share): 
    share_url = 'https://uzse.uz/isu_infos/STK?isu_cd='
    compiler = re.compile('\d+\s?,?\d*')
    print('Getting prices for', share['company'], end=' => ')
    r = requests.get(share_url+share['code'])
    cnt = BS(r.content, 'html.parser')
    prices = []
    for el in cnt.select('.conclusion-table .text-right'):
        price = el.select('.trd-price.price-up')
        if compiler.search(el.text) != None:
            index = compiler.search(el.text).span()
            price = float(el.text[index[0]:index[1]].replace(',', '.').replace(' ', ''))
            prices.append(price)
        else:
            prices.append(0)
    print(prices)
    return {'nominal': prices[0], 'base': prices[1], 'current': prices[2]}

# convert preprocessed data into actives list adding accumulating shares 
# if their company (Issue Code) is the same
def process_data(sort):
    actives = {}
    for trade in sort:
        active = {'Company': '', 'Quantity': 0, 'Total profit': 0, 'Total spent': 0, 'Base price': 0, 'Current price': 0, 'Price change': 0, 'Profit change': 0, 'Fee': 0, 'Trades': 0}
        if trade['Issue Code'] in actives:
            active = actives[trade['Issue Code']]
        active['Company'] = trade['Issue Name']
        active['Trades'] += 1
        active['Quantity'] += trade['Executed Quantity']
        active['Base price'] += trade['Execution Price']
        active['Total spent'] += trade['Trading Amount']
        active['Fee'] += trade['Fee']
        actives[trade['Issue Code']] = active

    for i in actives:
        share = actives[i]
        current = get_prices({'code': i, 'company': share['Company']})
        price = current['current']
        share['Current price'] = price
        share['Base price'] = share['Base price'] / share['Trades']
        share['Price change'] = price - share['Base price']
        share['Total profit'] = price * share['Quantity']
        share['Profit change'] = share['Total profit'] - share['Total spent']
    return actives

# print human readable rows
def e(str1, str2='', length=40):
    print(str1.ljust(length, ' '), str2)

# print human readable numbers like 123 000 000.00
def n(num, pad=20, dec=2, thousands=' ', decimal='.'):
    return ("{0:"+str(pad)+",."+str(dec)+"f}").format(num).replace(',', thousands).replace('.', decimal)

## Calculate the overall data

In [6]:
sort = preprocess_data(data)
actives = process_data(sort)
currentIndex = 0
baseIndex = 0
totalProfit = 0
nsa = 0
ta = 0
fee = 0
eq = 0
ep = 0
trades = len(sort)
c = collections.Counter()
for item in sort:
    nsa += item['Net Settlement Amount']
    ta += item['Trading Amount']
    fee += item['Fee']
    eq += item['Executed Quantity']
    ep += item['Execution Price']

for code, active in actives.items():
    currentIndex += active['Current price']
    baseIndex += active['Base price']
    totalProfit += active['Total profit']

Getting prices for O'zsanoatqurilishbank ATB => [19.0, 17.0, 17.0]
Getting prices for KAFOLAT Insurance Company JSC => [1.0, 0, 3.2]
Getting prices for Kvarts AJ => [1715.0, 3010.0, 3010.0]
Getting prices for Qizilqumsement AJ => [1570.0, 1860.0, 1860.0]
Getting prices for Hamkorbank ATB => [5.0, 25.5, 25.5]
Getting prices for Chilonzor buyum savdo kompleksi AJ => [10.0, 10.0, 10.0]
Getting prices for Agrobank ATB => [1168.0, 800.0, 800.0]
Getting prices for IPOTEKA-BANK ATIB => [1.0, 2.24, 2.24]
Getting prices for Hamkorbank ATB => [5.0, 74.99, 74.99]
Getting prices for IPOTEKA-BANK ATIB => [1.0, 0.9, 0.9]
Getting prices for Kapital sug'urta AJ => [1.0, 1.42, 1.42]


## Print

In [7]:
def print_data():
    sorted_actives = sorted(actives, key=lambda x: actives[x]['Total spent'], reverse=True)
    e('Umumiy harajatlar', n(nsa))
    e('Savdo harajatlari', n(ta))
    e('To\'lovlar', n(fee))
    e('Aktivlar qiymati', n(totalProfit))
    e('Aksiyalar soni', n(eq))
    e('Xarid narxlari indeksi', n(baseIndex / len(actives)))
    e('Joriy narxlar indeksi', n(currentIndex / len(actives)))
    print('-----------------------------')
    for i in sorted_actives:
        e(actives[i]['Company'], n(actives[i]['Profit change']) + n(actives[i]['Total spent']))
    print('-----------------------------')
    e( 'Daromad', n(totalProfit - ta))
    
print_data()

Umumiy harajatlar                               12 965 878.10
Savdo harajatlari                               12 837 503.05
To'lovlar                                          128 375.05
Aktivlar qiymati                                12 660 520.00
Aksiyalar soni                                   1 423 630.00
Xarid narxlari indeksi                                 545.32
Joriy narxlar indeksi                                  527.75
-----------------------------
Kvarts AJ                                           -3 589.00        2 983 489.00
Chilonzor buyum savdo kompleksi AJ                  14 665.10        2 885 334.90
Qizilqumsement AJ                                 -220 000.00        1 894 000.00
O'zsanoatqurilishbank ATB                           -3 890.00        1 533 890.00
IPOTEKA-BANK ATIB                                   10 000.00        1 222 000.00
Hamkorbank ATB                                      49 840.00        1 150 000.00
Hamkorbank ATB                              